In [7]:
import sys
sys.path.append('../data')
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb

In [9]:
cd ..

E:\gdrive\Projects\Porto_Seguro\stacking


In [4]:
# custom objective function (similar to auc)

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

In [10]:
train = pd.read_csv('data/train.csv', na_values=-1)
train = train.fillna(train.median())

In [11]:
train = train.drop(train.columns[train.columns.str.startswith('ps_calc') == True], axis=1)
train = train.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                    'ps_ind_18_bin','ps_car_03_cat', 'ps_car_10_cat'], axis=1)

In [13]:
train = pd.get_dummies(train, columns=train.columns[train.columns.str.endswith('cat')==True])

In [14]:
X = train.drop(['target','id'], axis=1)
y = train.target

In [18]:
test = pd.read_csv('data/test.csv', na_values=-1)
test = test.fillna(test.median())
test = test.drop(test.columns[test.columns.str.startswith('ps_calc') == True], axis=1)
test = test.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                    'ps_ind_18_bin','ps_car_03_cat', 'ps_car_10_cat'], axis=1)
test = pd.get_dummies(test, columns=test.columns[test.columns.str.endswith('cat')==True])
X_test = test.drop('id', axis=1)

In [21]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, shuffle=False, random_state=123)
# X = X.values
# y = y.values

In [9]:
# # create dataset for lightgbm
# lgb_train = lgb.Dataset(X_train, y_train)
# lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [16]:
params = {'device': 'gpu',
          'verbosity': -1, 
          'metric': 'auc', 
          'objective': 'binary', 
          'learning_rate': 0.01, 
          'max_depth': 6, 
          'min_gain_to_split': 0.1, 
          'min_sum_hessian_in_leaf': 7, 
          'poission_max_delta_step': 7, 
          'bagging_fraction': 0.9, 
          'feature_fraction': 0.6}

In [22]:
# from sklearn.externals import joblib
# model_num = 1
score = []
pred = pd.DataFrame({'id': test.id,'target': np.zeros_like(test.id)})
for train_index, val_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", val_index)
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    # Set Var for LGB
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)
    #
    lgb_skf = lgb.train(params, lgb_train, 10000, lgb_val, verbose_eval=100, 
                        feval=gini_lgb, early_stopping_rounds=100)
    print(lgb_skf.best_score['valid_0']['gini'])
    score.append(lgb_skf.best_score['valid_0']['gini'])
    pred_lap = lgb_skf.predict(X_test)
    pred['target'] += pred_lap 
#     joblib.dump(xgb_skf, 'model/xgb'+str(model_num)+'.pkl')
#     model_num += 1
pred['target'] = pred['target']/5

TRAIN: [197362 197453 197497 ..., 595209 595210 595211] TEST: [     0      1      2 ..., 198446 198447 198448]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.625278	valid_0's gini: 0.250555
[200]	valid_0's auc: 0.627382	valid_0's gini: 0.254764
[300]	valid_0's auc: 0.630757	valid_0's gini: 0.261513
[400]	valid_0's auc: 0.634341	valid_0's gini: 0.268682
[500]	valid_0's auc: 0.637056	valid_0's gini: 0.274113
[600]	valid_0's auc: 0.639352	valid_0's gini: 0.278704
[700]	valid_0's auc: 0.640866	valid_0's gini: 0.281732


KeyboardInterrupt: 

In [55]:
np.mean(score)

0.28351218481134333

In [59]:
pred.to_csv('sub/sub18.csv', index=False)

In [60]:
sub16 = pd.read_csv('sub/sub16.csv')

In [69]:
#combining xgb and lgb with geomean
pred_mix = pred
pred_mix['target'] = np.sqrt(sub16.target*pred.target)

In [70]:
pred_mix.to_csv('sub/sub19.csv', index=False)